## Import Libraries

In [1]:
from os import path, makedirs
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio

from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt
from brainpipe.visual import *
from brainpipe.statistics import *
from scipy.stats import *

## Power Decoding - Partial//Detailed Encoding
### For ALL time points

In [14]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.model_selection import permutation_test_score
from sklearn.metrics import roc_auc_score
from numpy.random import permutation

st = study('Olfacto')
reps = ['_Encoding_EpiPerf_LowHigh_mean_3s_bsl/','_Retrieval_EpiPerf_LowHigh_mean_3s_bsl/']
conds, subjects = ['low','high'],['VACJ','SEMC','LEFC','PIRJ','FERJ','CHAF']
freqs = ['theta','alpha','beta','gamma']
path_pow = path.join(st.path, 'feature/{}/{}_odor_{}_common_renamed_OFC_HC_pow.npz')
save_path = path.join(st.path, 'classified/0_Classif_Pow_sklearn_mean_{}_by_cond_3s/')
clf_name = path.join(save_path, '{}_LDA_clf_{}_{}_{}.npz')
nperm = 1000

for rep in reps:
    for su in subjects:
        for i,freq in enumerate(freqs):
            pow_list = []
            #=========================== Load Power files (nfreq, nelec, nwin, ntrial) =================================    
            mat0 = np.load(path_pow.format(rep,su,conds[0]), allow_pickle=True)
            pow_list.append(np.mean(mat0['xpow_n'][i,...],axis=1))
            nelecs = mat0['xpow_n'][i,...].shape[0]
            mat1 = np.load(path_pow.format(rep,su,conds[1]), allow_pickle=True)
            pow_list.append(np.mean(mat1['xpow_n'][i,...],axis=1))
            print (su,rep,mat0.files, 'xpow shape: ', [pow.shape for pow in pow_list])

            #=========================== Create dict for all results =================================    
            name_classif = clf_name.format(rep[1],su,freq,conds[0],conds[1])
            if not path.exists(name_classif):
                kwargs = {}
                kwargs['names'], kwargs['channels'] = mat0['labels'], mat0['channels']
                kwargs['xyz'] = mat0['xyz']

                # =========================== Select Power for 1 elec 1 freq =================================                 
                permut,auc = np.zeros((nelecs,nperm)),np.zeros((nelecs))
                for elec_num in range(nelecs):
                    print('--» processing',su, 'elec', elec_num,'/',nelecs, 'freq',freq)
                    pow_data_elec = [power[elec_num][:,np.newaxis] for power in pow_list]

                    # create a data matrix, concatenate along the trial dimension
                    x = np.concatenate(pow_data_elec, axis=0)
                    print ('Size of the concatenated data: ', x.shape)
                    y = np.hstack([np.array([i]*len(power)) for i, power in enumerate(pow_data_elec)])
                    print ('Size of label for classif: ', len(y))

                    clf = LDA()
                    cv = SKFold(5)
                    score, permutation_scores, pvalue = permutation_test_score(
                    clf, x, y, scoring="roc_auc", cv=cv, n_permutations=nperm, n_jobs=-1)
                    permut[elec_num] += permutation_scores
                    auc[elec_num] += score
                    print("Classification score %s (pvalue : %s)" % (score, pvalue))

                kwargs['perm'], kwargs['auc'] = permut, auc
                #Save plots
                np.savez(name_classif, **kwargs)
                del x, y, pow_data_elec, permutation_scores, score
            else:
                print(name_classif, 'already computed')
        del pow_list

-> Olfacto loaded
VACJ _Encoding_EpiPerf_LowHigh_mean_3s_bsl/ ['xpow_od', 'xpow_bsl', 'xpow_n', 'f', 'split', 'width', 'step', 'fnames', 'time', 'labels', 'xyz', 'channels'] xpow shape:  [(48, 11), (48, 11)]
--» processing VACJ elec 0 / 48 freq theta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.75 (pvalue : 0.06293706293706294)
--» processing VACJ elec 1 / 48 freq theta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.861111111111111 (pvalue : 0.004995004995004995)
--» processing VACJ elec 2 / 48 freq theta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.5277777777777778 (pvalue : 0.3886113886113886)
--» processing VACJ elec 3 / 48 freq theta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.45 (pvalue : 0.5084915084915085)
--» processing VACJ elec 4 / 48 freq theta
Size of the concatenated data:  (22, 1)

Classification score 0.8 (pvalue : 0.02197802197802198)
--» processing VACJ elec 45 / 48 freq theta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.6777777777777778 (pvalue : 0.23576423576423577)
--» processing VACJ elec 46 / 48 freq theta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.3277777777777778 (pvalue : 0.8171828171828172)
--» processing VACJ elec 47 / 48 freq theta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.1 (pvalue : 0.988011988011988)
VACJ _Encoding_EpiPerf_LowHigh_mean_3s_bsl/ ['xpow_od', 'xpow_bsl', 'xpow_n', 'f', 'split', 'width', 'step', 'fnames', 'time', 'labels', 'xyz', 'channels'] xpow shape:  [(48, 11), (48, 11)]
--» processing VACJ elec 0 / 48 freq alpha
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.8277777777777778 (pvalue : 0.011988011988011988)
--» processing VACJ elec 1 /

Classification score 0.6 (pvalue : 0.3676323676323676)
--» processing VACJ elec 41 / 48 freq alpha
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.5944444444444444 (pvalue : 0.43756243756243757)
--» processing VACJ elec 42 / 48 freq alpha
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.11666666666666665 (pvalue : 0.986013986013986)
--» processing VACJ elec 43 / 48 freq alpha
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.5833333333333333 (pvalue : 0.3986013986013986)
--» processing VACJ elec 44 / 48 freq alpha
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.6055555555555555 (pvalue : 0.35564435564435565)
--» processing VACJ elec 45 / 48 freq alpha
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.75 (pvalue : 0.06893106893106893)
--» processing VACJ elec 46 / 4

Classification score 0.6277777777777778 (pvalue : 0.2957042957042957)
--» processing VACJ elec 38 / 48 freq beta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.34444444444444444 (pvalue : 0.6843156843156843)
--» processing VACJ elec 39 / 48 freq beta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.8 (pvalue : 0.07792207792207792)
--» processing VACJ elec 40 / 48 freq beta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.5333333333333334 (pvalue : 0.4675324675324675)
--» processing VACJ elec 41 / 48 freq beta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.35 (pvalue : 0.7282717282717283)
--» processing VACJ elec 42 / 48 freq beta
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.4833333333333333 (pvalue : 0.46153846153846156)
--» processing VACJ elec 43 / 48 fre

Classification score 0.8166666666666668 (pvalue : 0.028971028971028972)
--» processing VACJ elec 35 / 48 freq gamma
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.6 (pvalue : 0.38161838161838163)
--» processing VACJ elec 36 / 48 freq gamma
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.2166666666666667 (pvalue : 0.9230769230769231)
--» processing VACJ elec 37 / 48 freq gamma
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.2611111111111111 (pvalue : 0.8711288711288712)
--» processing VACJ elec 38 / 48 freq gamma
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.3 (pvalue : 0.8231768231768232)
--» processing VACJ elec 39 / 48 freq gamma
Size of the concatenated data:  (22, 1)
Size of label for classif:  22
Classification score 0.55 (pvalue : 0.4825174825174825)
--» processing VACJ elec 40 / 48 freq gamma
Si

...

Classification score 0.5722222222222222 (pvalue : 0.42357642357642356)
--» processing CHAF elec 46 / 57 freq theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.4611111111111111 (pvalue : 0.5654345654345654)
--» processing CHAF elec 47 / 57 freq theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.3555555555555555 (pvalue : 0.7372627372627373)
--» processing CHAF elec 48 / 57 freq theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.16111111111111112 (pvalue : 0.965034965034965)
--» processing CHAF elec 49 / 57 freq theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6055555555555555 (pvalue : 0.40559440559440557)
--» processing CHAF elec 50 / 57 freq theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.3111111111111111 (pvalue : 0.8321678321678322)
--»

Classification score 0.25555555555555554 (pvalue : 0.8571428571428571)
--» processing CHAF elec 33 / 57 freq alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.7222222222222222 (pvalue : 0.1028971028971029)
--» processing CHAF elec 34 / 57 freq alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.11666666666666665 (pvalue : 0.981018981018981)
--» processing CHAF elec 35 / 57 freq alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.8055555555555556 (pvalue : 0.01898101898101898)
--» processing CHAF elec 36 / 57 freq alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.8111111111111111 (pvalue : 0.04495504495504495)
--» processing CHAF elec 37 / 57 freq alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.1388888888888889 (pvalue : 0.988011988011988)
--»

Classification score 0.8388888888888889 (pvalue : 0.014985014985014986)
--» processing CHAF elec 20 / 57 freq beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.861111111111111 (pvalue : 0.013986013986013986)
--» processing CHAF elec 21 / 57 freq beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.55 (pvalue : 0.4935064935064935)
--» processing CHAF elec 22 / 57 freq beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.06666666666666667 (pvalue : 1.0)
--» processing CHAF elec 23 / 57 freq beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.16666666666666669 (pvalue : 0.9400599400599401)
--» processing CHAF elec 24 / 57 freq beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.711111111111111 (pvalue : 0.11788211788211789)
--» processing CHAF elec 25 / 57 f

Classification score 0.6166666666666667 (pvalue : 0.34965034965034963)
--» processing CHAF elec 7 / 57 freq gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.2666666666666667 (pvalue : 0.8791208791208791)
--» processing CHAF elec 8 / 57 freq gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.7055555555555555 (pvalue : 0.11988011988011989)
--» processing CHAF elec 9 / 57 freq gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.3111111111111111 (pvalue : 0.8401598401598401)
--» processing CHAF elec 10 / 57 freq gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.19444444444444445 (pvalue : 0.977022977022977)
--» processing CHAF elec 11 / 57 freq gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.5777777777777777 (pvalue : 0.4035964035964036)
--» pr

Classification score 0.9 (pvalue : 0.011988011988011988)
--» processing CHAF elec 52 / 57 freq gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.45555555555555555 (pvalue : 0.5894105894105894)
--» processing CHAF elec 53 / 57 freq gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.23333333333333334 (pvalue : 0.8821178821178821)
--» processing CHAF elec 54 / 57 freq gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.33333333333333337 (pvalue : 0.6863136863136863)
--» processing CHAF elec 55 / 57 freq gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.2833333333333333 (pvalue : 0.7872127872127872)
--» processing CHAF elec 56 / 57 freq gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.11666666666666667 (pvalue : 0.99000999000999)
